In [1]:
using Revise

In [221]:
using QuantumStates
using LinearAlgebra
using Plots

In [3]:
const c = 299792458.0
const h = 6.626e-34
const ħ = h / 2π
;

### Define X state

In [4]:
# Spectroscopic constants
# From Scurlock et al. 
const BX = 10023.0841
const DX = 1.154e-2 
const γX = 34.7593 
const bFX = 2.602 
const cX = 2.053 
const μX = 1e-6 * 1.465 * 10^2 * (1e-21 / c) / h # 1e-6 = Hz to MHz
;

In [64]:
QN_bounds = (S = 1/2, I = 1/2, Λ = -1:1, N = 0:2)
basis_X = enumerate_states(HundsCaseB_Rot, QN_bounds)
ordered_basis_X = order(basis_X, :M)
;

In [65]:
# Define the zero-field Hamiltonian operator
H_X_operator(state, state′, i, j, p) = 
    (BX * Rotation + 
    DX * RotationDistortion + 
    γX * SpinRotation + 
    bFX * Hyperfine_IS + 
    (cX/3) * Hyperfine_Dipolar)(state, state′)
;

In [66]:
p = NamedTuple()
H_X = Hamiltonian(H_operator=H_X_operator, basis=ordered_basis_X)
update_matrix!(H_X, p)
solve!(H_X)
;

### Define A state

In [67]:
# Spectroscopic constants for the A state
# From Li & Coxon (1995)
const BA = 0.3412200 * c * 1e2
const AA = 66.8181 * c * 1e2
const pA = -0.04287 * c * 1e2
const qA = -0.3257e-3 * c * 1e2
const aA = 1.0 * 1e4
const bFA = 1.0 * 1e4
const cA = 1.0 * 1e4
const dA = 1.0 * 1e4
;

In [256]:
# Enumerate all states of Λ = -1 in a Hund's case (a) basis
QN_bounds = (S = 1/2, I = 1/2, Λ = -1:1, J = (1/2, 3/2))
basis_A = enumerate_states(HundsCaseA_Rot, QN_bounds)
ordered_basis_A = order(basis_A, :M)
;

In [257]:
# Define the zero-field Hamiltonian operator
H_A_operator(state, state′, i, j, p) = 
    (BA * Rotation +
    AA * SpinOrbit + 
    BA * SpinUncoupling + 
    qA * ΛDoubling_q +
    (pA + 2qA) * ΛDoubling_p2q)(state, state′) #+
#     aA * Hyperfine_IL +
#     bFA * Hyperfine_IF +
#     cA * Hyperfine_Dipolar_c +
#     dA * Hyperfine_Dipolar_d
;

In [258]:
p = NamedTuple()
H_A = Hamiltonian(H_operator=H_A_operator, basis=ordered_basis_A)
update_matrix!(H_A, p)
solve!(H_A)
;

### Convert A state from Hund's case (a) to case (b) to evaluate TDMs

In [259]:
A_states_caseB = convert_basis(H_A.states, ordered_basis_X);

In [265]:
# A state that we're coupling through
H_A.states[6] |> display
A_states_caseB[6] |> display

State{HundsCaseA_Rot}(-9.906962342685342e11, HundsCaseA_Rot[HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, -1//1, 3//2, -3//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -1.5:1.0:-1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, 0//1, 3//2, -1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -0.5:1.0:-0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, 1//1, 3//2, 1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 0.5:1.0:0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, -1//1, 3//2, -1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -0.5:1.0:-0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, 0//1, 3//2, 1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 0.5:1.0:0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, 1//1, 3//2, 3//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 1.5:1.0:1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, -1//1, 3//2, -3//2, 1//1, -1//1, (Σ

State{HundsCaseB_Rot}(-9.906962342685342e11, HundsCaseB_Rot[HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 3//1, -3//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 0//1, 2//1, 5//2, 3//1, -3//1, (N = 0:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 1//1, 2//1, 5//2, 3//1, -3//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 1//1, 3//2, 2//1, -2//1, (N = 1:∞, J = 0.5:1.0:1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 3//2, 2//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 2//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 3//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 0//1, 1//1, 3/

In [267]:
# X state that we're going from (1)
H_X.states[32]

State{HundsCaseB_Rot}(20011.336392203437, HundsCaseB_Rot[HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 3//1, -3//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 0//1, 2//1, 5//2, 3//1, -3//1, (N = 0:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 1//1, 2//1, 5//2, 3//1, -3//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 1//1, 3//2, 2//1, -2//1, (N = 1:∞, J = 0.5:1.0:1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 3//2, 2//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 2//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 3//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 0//1, 1//1, 3//2,

In [268]:
# X state that we're going from (2)
H_X.states[40]

State{HundsCaseB_Rot}(20064.083756666663, HundsCaseB_Rot[HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 3//1, -3//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 0//1, 2//1, 5//2, 3//1, -3//1, (N = 0:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 1//1, 2//1, 5//2, 3//1, -3//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 1//1, 3//2, 2//1, -2//1, (N = 1:∞, J = 0.5:1.0:1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 3//2, 2//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 2//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -2.0:1.0:2.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, -1//1, 2//1, 5//2, 3//1, -2//1, (N = 1:∞, J = 1.5:1.0:2.5, F = 2.0:1.0:3.0, M = -3.0:1.0:3.0)), HundsCaseB_Rot(0.0, 1//2, 1//2, 0//1, 1//1, 3//2,

In [269]:
TDM(H_X.states[32], A_states_caseB[6], -1)^2

0.2263361938462067 + 0.0im

In [270]:
TDM(H_X.states[40], A_states_caseB[6], +1)^2

0.16666666666666652 - 0.0im

### Convert X state from Hund's case (b) to case (a) to evaluate TDMs

In [271]:
X_states_caseA = convert_basis(H_X.states, ordered_basis_A);

In [272]:
X_states_caseA[32]

State{HundsCaseA_Rot}(20011.336392203437, HundsCaseA_Rot[HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, -1//1, 3//2, -3//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -1.5:1.0:-1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, 0//1, 3//2, -1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -0.5:1.0:-0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, 1//1, 3//2, 1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 0.5:1.0:0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, -1//1, 3//2, -1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -0.5:1.0:-0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, 0//1, 3//2, 1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 0.5:1.0:0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, 1//1, 3//2, 3//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 1.5:1.0:1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, -1//1, 3//2, -3//2, 1//1, -1//1, (Σ = 

In [273]:
X_states_caseA[40]

State{HundsCaseA_Rot}(20064.083756666663, HundsCaseA_Rot[HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, -1//1, 3//2, -3//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -1.5:1.0:-1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, 0//1, 3//2, -1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -0.5:1.0:-0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, 1//1, 3//2, 1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 0.5:1.0:0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, -1//1, 3//2, -1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = -0.5:1.0:-0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, 0//1, 3//2, 1//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 0.5:1.0:0.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, 1//2, 1//1, 3//2, 3//2, 2//1, -2//1, (Σ = -0.5:1.0:0.5, Ω = 1.5:1.0:1.5, F = 1.0:1.0:2.0, M = -2.0:1.0:2.0)), HundsCaseA_Rot(0.0, 1//2, 1//2, -1//2, -1//1, 3//2, -3//2, 1//1, -1//1, (Σ = 

In [280]:
TDM(X_states_caseA[32], H_A.states[6], 1)^2

0.2263361938462067 + 0.0im

In [279]:
TDM(X_states_caseA[40], H_A.states[6], -1)^2

0.16666666666666666 - 0.0im

### Estimate shifts for each of the two states

In [309]:
ε₀ = 8.8541878128 * 10^(-12)
ħ = 1.054571817 * 10^(-34)
ω = 2π * c / 626e-9
c0 = 299792458.0
;

In [310]:
effective_n_beams = 1
I = effective_n_beams * 16 * 10 # factor 10 to go from mW/cm^2 to W/m^2
;

In [311]:
Ω_squared = (2I / (c * ε₀)) * (d_squared / ħ^2)

3.33668534907267e13

In [319]:
# |J=1/2, F=1, m=0⟩ shift
# Factors are: 2/3 --> intensity ratio for J = 1/2
d_squared = (2/3) * (0.22633619384620707) * (1 / (2π * 25e-9)) * ((3π * ε₀ * ħ * c0^3) / (ω^3)) # 2π needed?
Ω_squared = (2I / (c0 * ε₀)) * (d_squared / ħ^2)
Ω_squared / (4 * 14e6) |> display

# Cross-coupling shift
Ω_squared / (4 * 52.75e6) |> display

1.6183128470818042e6

429504.83145299065

In [320]:
# |J=3/2, F=2, m=2⟩ shift
# Factors are: 1/3 --> intensity ratio for J = 3/2
d_squared = (1/3) * (0.16666666666666716) * (1 / (2π * 25e-9)) * ((3π * ε₀ * ħ * c0^3) / (ω^3)) # 2π needed?
Ω_squared = (2I / (c0 * ε₀)) * (d_squared / ħ^2)
Ω_squared / (4 * 14e6)

595836.6694772625